# Prelimary analysis

In [ ]:
import pandas as pd


df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove Whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

print(df.columns)

### City stats

#### This code calculates the total funding amount, total population, median household income, and percentage of the population identifying as White for each city. It groups the data by city, computes funding per capita, and sorts the cities in descending order based on funding per capita, displaying the top results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")

# Remove whitespace to clean data
df = df.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)

# Group by 'City' to get aggregated data
city_stats = df.groupby('City').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max', 
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Calculate funding per capita
city_stats['Funding_Per_Capita'] = city_stats['Funding Amount Excluding Loans'] / city_stats['Total_Population']

# Calculate percentage of White population
city_stats['Percentage_White'] = (city_stats['White_Alone'] / city_stats['Total_Population']) * 100

# Rearrange columns to place Funding_Per_Capita next to City
city_stats = city_stats[[
    'City', 
    'Funding_Per_Capita',
    'Funding Amount Excluding Loans',
    'Total_Population', 
    'White_Alone', 
    'Percentage_White', 
    'Median_household_income'
]]

# Sort by City in ascending order
sorted_city_stats = city_stats.sort_values(by='City', ascending=True).reset_index(drop=True)

# Ensure all columns print side by side
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Display the final DataFrame
display(sorted_city_stats)

# Sort cities by Funding_Per_Capita in descending order
sorted_city_stats = city_stats.sort_values(by='Funding (in Millions)', ascending=False).reset_index(drop=True)
print(sorted_city_stats[['City', 'Funding (in Millions)', 'Total_Population', 'Funding_Per_Capita']])



# Maps

## Get Coordinates for each county ##

In [ ]:
from geopy.geocoders import Nominatim
import time

# List of unique counties
counties = list(df['County'].unique())

# Create dictionary to store county coordinates
geolocator = Nominatim(user_agent="massachusetts_counties")
county_coordinates = {}

print("Fetching coordinates for counties...")
for county in counties:
    try:
        location = geolocator.geocode(f"{county}, Massachusetts")
        if location:
            county_coordinates[county] = (location.latitude, location.longitude)
        else:
            city_coordinates[county] = None
        time.sleep(1)  # Avoid hitting the API rate limit
    except Exception as e:
        print(f"Error fetching coordinates for {county}: {e}")
        county_coordinates[county] = None

# Print or save the county coordinates
print(county_coordinates)

# Optionally save to a CSV file
coordinates_df = pd.DataFrame.from_dict(county_coordinates, orient='index', columns=['Latitude', 'Longitude'])
coordinates_df.reset_index(inplace=True)
coordinates_df.rename(columns={'index': 'County'}, inplace=True)
coordinates_df.to_csv('massachusetts_county_coordinates.csv', index=False)

print("Coordinates fetched and saved to 'massachusetts_county_coordinates.csv'.")


### Heat Map by county ###

In [ ]:
import pandas as pd
import folium
import json
import requests
from folium.features import GeoJsonTooltip
from IPython.display import display

df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove Whitespace to clean data
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)


# Population data from https://malegislature.gov/Redistricting/MassachusettsCensusData/County
county_coordinates = {
    "Plymouth": (41.9426657, -70.7618592),
    "Middlesex": (42.485452, -71.3968261),
    "Hampshire": (42.3432499, -72.6213339),
    "Essex": (42.6320389, -70.7828255),
    "Dukes": (41.3926378, -70.642011),
    "Bristol": (41.7425538, -71.0856545),
    "Barnstable": (41.7016936, -70.3036163),
    "Worcester": (42.2625621, -71.8018877),
    "Norfolk": (42.1538607, -71.1828015),
    "Suffolk": (42.3544455, -70.9788771),
    "Franklin": (42.5896205, -72.6110645),
    "Hampden": (42.1285315, -72.6063441),
    "Berkshire": (42.3999954, -73.2322639),
    "Nantucket": (41.2727997, -70.0951867)
}

county_population = {
    "Plymouth": 530819,
    "Middlesex": 1632002,
    "Hampshire": 162308,
    "Essex": 809829,
    "Dukes": 20600,
    "Bristol": 579200,
    "Barnstable": 228996,
    "Worcester": 862111,
    "Norfolk": 725981,
    "Suffolk": 797936,
    "Franklin": 71029,
    "Hampden": 465825,
    "Berkshire": 129026,
    "Nantucket": 14255
}

# Group by 'County' to calculate funding and income
#county_stats = df[df['County'].isin(county_coordinates.keys())].groupby('County').agg({
county_stats = df.groupby('County').agg({
    'Funding Amount Excluding Loans': 'sum',
}).reset_index()

# Add population data from the dictionary
county_stats['Total_Population'] = county_stats['County'].map(county_population)

# Calculate funding per capita
county_stats['Funding_Per_Capita'] = county_stats['Funding Amount Excluding Loans'] / county_stats['Total_Population']

# Normalize funding for better visualization
county_stats['Funding (in Millions)'] = county_stats['Funding Amount Excluding Loans'] / 1e6

# GetGeoJSON data from the URL
geojson_url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
response = requests.get(geojson_url)
geojson_data = response.json()

# Filter GeoJSON for Massachusetts counties only(Mass is state number 25)
massachusetts_geojson_features = [
    feature for feature in geojson_data['features']
    if feature['properties']['NAME'] in county_coordinates.keys()
    and feature['properties'].get('STATE') == "25"  
]
geojson_data['features'] = massachusetts_geojson_features



# Make dictionary for county funding data
funding_data = {
    row['County']: {
        "funding": row['Funding (in Millions)'],
        "population": row['Total_Population'],
        "funding_per_capita": row['Funding_Per_Capita']
    }
    for _, row in county_stats.iterrows()
}

# Add funding, population, and funding per capita to the GeoJSON
for feature in geojson_data['features']:
    county_name = feature['properties']['NAME']
    if county_name in funding_data:
        feature['properties']['Funding'] = funding_data[county_name]['funding']
        feature['properties']['Population'] = funding_data[county_name]['population']
        feature['properties']['Funding_Per_Capita'] = funding_data[county_name]['funding_per_capita']  # Add funding per capita
    else:
        feature['properties']['Funding'] = None
        feature['properties']['Population'] = None
        feature['properties']['Funding_Per_Capita'] = None  # Set funding per capita to None if not found


# Create the base map
ma_map = folium.Map(location=[42.4072, -71.3824], zoom_start=8)

# Create a Choropleth layer for funding distribution
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    data=county_stats,
    columns=["County", "Funding_Per_Capita"],
    key_on="feature.properties.NAME",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Funding Per Capita (USD)",
    threshold_scale=[1, 10, 50, 100, 200, 300, 400, 600, 900, 1200]
).add_to(ma_map)

# Add tooltips with hover information
tooltip = GeoJsonTooltip(
    fields=["NAME", "Funding", "Population", "Funding_Per_Capita"],
    aliases=["County:", "Funding (Millions):", "Population:", "Funding Per Capita (USD):"],
    localize=True,
    sticky=True,
    labels=True,
    style=(
        "background-color: white; "
        "border: 1px solid black; "
        "border-radius: 3px; "
        "padding: 5px;"
    )
)


# Add GeoJSON layer for interactivity
folium.GeoJson(
    geojson_data,
    style_function=lambda x: {
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.2,
    },
    tooltip=tooltip
).add_to(ma_map)

display(ma_map)
# Sort cities by Funding_Per_Capita in descending order
sorted_county_stats = county_stats.sort_values(by='Funding (in Millions)', ascending=False).reset_index(drop=True)

print(sorted_county_stats[['County', 'Funding (in Millions)', 'Total_Population', 'Funding_Per_Capita',]])



## Get coordintes for city ##

In [ ]:
from geopy.geocoders import Nominatim
import time

# List of unique counties
counties = list(df['City'].unique())

# Create dictionary to store county coordinates
geolocator = Nominatim(user_agent="massachusetts_cities")
city_coordinates = {}

print("Fetching coordinates for cities...")
for city in cities:
    try:
        location = geolocator.geocode(f"{city}, Massachusetts")
        if location:
            city_coordinates[city] = (location.latitude, location.longitude)
        else:
            city_coordinates[city] = None
        time.sleep(1)  # Avoid hitting the API rate limit
    except Exception as e:
        print(f"Error fetching coordinates for {city}: {e}")
        city_coordinates[city] = None

# Print or save the city coordinates
print(city_coordinates)

# Optionally save to a CSV file
coordinates_df = pd.DataFrame.from_dict(city_coordinates, orient='index', columns=['Latitude', 'Longitude'])
coordinates_df.reset_index(inplace=True)
coordinates_df.rename(columns={'index': 'city'}, inplace=True)
coordinates_df.to_csv('massachusetts_city_coordinates.csv', index=False)

print("Coordinates fetched and saved to 'massachusetts_city_coordinates.csv'.")

### Heat map by City per capita funding ###

In [ ]:
import pandas as pd
import folium
import requests
from folium.features import GeoJsonTooltip
from IPython.display import display
import numpy as np

# Load funding data
df = pd.read_csv("https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

# Group by 'City' to get aggregated data
city_stats = df.groupby('City').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max',
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Calculate funding per capita (handle division by zero)
city_stats['Funding_Per_Capita'] = city_stats.apply(
    lambda row: row['Funding Amount Excluding Loans'] / row['Total_Population']
    if row['Total_Population'] > 0 else 0, axis=1
)

# Calculate funding per capita (handle division by zero)
city_stats['Percentage_White'] = city_stats.apply(
    lambda row: row['White_Alone'] / row['Total_Population']
    if row['Total_Population'] > 0 else 0, axis=1
)


# Normalize funding for better visualization
city_stats['Funding (in Millions)'] = city_stats['Funding Amount Excluding Loans'] / 1e6

# Avoid taking the log of zero; add a small constant
city_stats['Log_Funding_Per_Capita'] = np.log1p(city_stats['Funding_Per_Capita'])  # log(1 + x)

# Fetch the GeoJSON file
geojson_url = "https://raw.githubusercontent.com/nblmc/massachusetts-municipal-boundaries/main/data-cooked/mass-municipalities.geojson"
response = requests.get(geojson_url)
geojson_data = response.json()

# Merge funding data into GeoJSON
for feature in geojson_data['features']:
    city_name = feature['properties']['census_name']
    city_row = city_stats[city_stats['City'].str.lower() == city_name.lower()]

    if not city_row.empty:
        # Populate from the grant funding CSV
        feature['properties']['Funding'] = float(city_row['Funding (in Millions)'].values[0])
        feature['properties']['Population'] = (
            int(city_row['Total_Population'].values[0]) if not np.isnan(city_row['Total_Population'].values[0]) else "Not Available"
        )
        feature['properties']['Funding_Per_Capita'] = float(city_row['Funding_Per_Capita'].values[0])
        feature['properties']['Log_Funding_Per_Capita'] = float(city_row['Log_Funding_Per_Capita'].values[0])  # Add Log column
        feature['properties']['Percentage_White'] = float(city_row['Percentage_White'].values[0])
        feature['properties']['Median_household_income'] = float(city_row['Median_household_income'].values[0])
    else:
        # Default values for cities not in the funding data
        feature['properties']['Funding'] = 0.0  # No funding
        feature['properties']['Population'] = "Not Available"  # Missing population
        feature['properties']['Funding_Per_Capita'] = 0.0  # No funding per capita
        feature['properties']['Log_Funding_Per_Capita'] = 0.0  # Default log funding per capita
        feature['properties']['Percentage_White'] = "Not Available"
        feature['properties']['Median_household_income'] = "Not Available"

# Dynamically calculate bins for Funding_Per_Capita
min_value = city_stats['Funding_Per_Capita'].min()
max_value = city_stats['Funding_Per_Capita'].max()
bins = np.linspace(min_value, max_value, num=10).tolist()
# print("Min_value:", min_value, "Max_value:", max_value)

# Create the base map
ma_map = folium.Map(location=[42.4072, -71.3824], zoom_start=8)

# Create a Choropleth layer for funding distribution
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    data=city_stats,
    columns=["City", "Log_Funding_Per_Capita"],
    key_on="feature.properties.census_name",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Log of Funding Per Capita (Log(1+x))",
).add_to(ma_map)

# Add tooltips with formatted funding per capita
tooltip = GeoJsonTooltip(
    fields=["census_name", "Funding", "Population", "Funding_Per_Capita", "Percentage_White", "Median_household_income"],
    aliases=["City:", "Funding (Millions):", "Population:", "Funding Per Capita (USD):", "Percentage White:", "Median Household Income (USD):"],
    localize=True,
    sticky=True,
    labels=True,
    style=(
        "background-color: white; "
        "border: 1px solid black; "
        "border-radius: 3px; "
        "padding: 5px;"
    ),
    missing="Not Available",
)
# Add GeoJSON layer for interactivity and coloring
folium.GeoJson(
    geojson_data,
    style_function=lambda x: {
        'fillColor': '#fee5d9' if x['properties']['Funding_Per_Capita'] == 0 else
                     '#fcae91' if x['properties']['Log_Funding_Per_Capita'] <= 1 else
                     '#fb6a4a' if x['properties']['Log_Funding_Per_Capita'] <= 2 else
                     '#de2d26' if x['properties']['Log_Funding_Per_Capita'] <= 3 else
                     '#a50f15',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=tooltip
).add_to(ma_map)

# Display the map
display(ma_map)

### Heat map by percent white

In [ ]:
import pandas as pd
import folium
import requests
from folium.features import GeoJsonTooltip
from IPython.display import display
import numpy as np

# Load funding data
df = pd.read_csv("https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

# Group by 'City' to get aggregated data
city_stats = df.groupby('City').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max',
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Calculate funding per capita (handle division by zero)
city_stats['Funding_Per_Capita'] = city_stats.apply(
    lambda row: row['Funding Amount Excluding Loans'] / row['Total_Population']
    if row['Total_Population'] > 0 else 0, axis=1
)

# Calculate funding per capita (handle division by zero)
city_stats['Percentage_White'] = city_stats.apply(
    lambda row: row['White_Alone'] / row['Total_Population']
    if row['Total_Population'] > 0 else 0, axis=1
)


# Normalize funding for better visualization
city_stats['Funding (in Millions)'] = city_stats['Funding Amount Excluding Loans'] / 1e6

# Avoid taking the log of zero; add a small constant
city_stats['Log_Funding_Per_Capita'] = np.log1p(city_stats['Funding_Per_Capita'])  # log(1 + x)

# Fetch the GeoJSON file
geojson_url = "https://raw.githubusercontent.com/nblmc/massachusetts-municipal-boundaries/main/data-cooked/mass-municipalities.geojson"
response = requests.get(geojson_url)
geojson_data = response.json()

# Merge funding data into GeoJSON
for feature in geojson_data['features']:
    city_name = feature['properties']['census_name']
    city_row = city_stats[city_stats['City'].str.lower() == city_name.lower()]

    if not city_row.empty:
        # Populate from the grant funding CSV
        feature['properties']['Funding'] = float(city_row['Funding (in Millions)'].values[0])
        feature['properties']['Population'] = (
            int(city_row['Total_Population'].values[0]) if not np.isnan(city_row['Total_Population'].values[0]) else "Not Available"
        )
        feature['properties']['Funding_Per_Capita'] = float(city_row['Funding_Per_Capita'].values[0])
        feature['properties']['Log_Funding_Per_Capita'] = float(city_row['Log_Funding_Per_Capita'].values[0])  # Add Log column
        feature['properties']['Percentage_White'] = float(city_row['Percentage_White'].values[0])
        feature['properties']['Median_household_income'] = float(city_row['Median_household_income'].values[0])
    else:
        # Default values for cities not in the funding data
        feature['properties']['Funding'] = 0.0  # No funding
        feature['properties']['Population'] = "Not Available"  # Missing population
        feature['properties']['Funding_Per_Capita'] = 0.0  # No funding per capita
        feature['properties']['Log_Funding_Per_Capita'] = 0.0  # Default log funding per capita
        feature['properties']['Percentage_White'] = "Not Available"
        feature['properties']['Median_household_income'] = "Not Available"

# Dynamically calculate bins for Percentage White
min_value = city_stats['Percentage_White'].min()
max_value = city_stats['Percentage_White'].max()
bins = np.linspace(min_value, max_value, num=6).tolist()
# print("Min_value:", min_value, "Max_value:", max_value)

# Create the base map
ma_map = folium.Map(location=[42.4072, -71.3824], zoom_start=8)

# Create a Choropleth layer for funding distribution
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    data=city_stats,
    columns=["City", "Percentage_White"],
    key_on="feature.properties.census_name",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Percentage White",
    threshold_scale=bins
).add_to(ma_map)

# Add tooltips with formatted funding per capita
tooltip = GeoJsonTooltip(
    fields=["census_name", "Funding", "Population", "Funding_Per_Capita", "Percentage_White", "Median_household_income"],
    aliases=["City:", "Funding (Millions):", "Population:", "Funding Per Capita (USD):", "Percentage White:", "Median Household Income (USD):"],
    localize=True,
    sticky=True,
    labels=True,
    style=(
        "background-color: white; "
        "border: 1px solid black; "
        "border-radius: 3px; "
        "padding: 5px;"
    ),
    missing="Not Available",
)
# Add GeoJSON layer for interactivity and coloring
folium.GeoJson(
    geojson_data,
    style_function=lambda x: {
        'fillColor': '#fee5d9' if x['properties']['Percentage_White'] == "Not Available" else
                     '#f7f4f9' if bins[0] <= x['properties']['Percentage_White'] < bins[1] else
                     '#d0d1e6' if bins[1] <= x['properties']['Percentage_White'] < bins[2] else
                     '#a6bddb' if bins[2] <= x['properties']['Percentage_White'] < bins[3] else
                     '#3690c0' if bins[3] <= x['properties']['Percentage_White'] < bins[4] else
                     '#034e7b',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=tooltip
).add_to(ma_map)

# Display the map
display(ma_map)

### Heat map by median household income

In [ ]:
import pandas as pd
import folium
import requests
from folium.features import GeoJsonTooltip
from IPython.display import display
import numpy as np

# Load funding data
df = pd.read_csv("https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

# Group by 'City' to get aggregated data
city_stats = df.groupby('City').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max',
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Calculate funding per capita (handle division by zero)
city_stats['Funding_Per_Capita'] = city_stats.apply(
    lambda row: row['Funding Amount Excluding Loans'] / row['Total_Population']
    if row['Total_Population'] > 0 else 0, axis=1
)

# Calculate median household income bins
min_income = city_stats['Median_household_income'].min()
max_income = city_stats['Median_household_income'].max()
income_bins = np.linspace(min_income, max_income, num=6).tolist()

# Fetch the GeoJSON file
geojson_url = "https://raw.githubusercontent.com/nblmc/massachusetts-municipal-boundaries/main/data-cooked/mass-municipalities.geojson"
response = requests.get(geojson_url)
geojson_data = response.json()

# Merge funding data into GeoJSON
for feature in geojson_data['features']:
    city_name = feature['properties']['census_name']
    city_row = city_stats[city_stats['City'].str.lower() == city_name.lower()]

    if not city_row.empty:
        # Populate from the grant funding CSV
        feature['properties']['Funding'] = float(city_row['Funding Amount Excluding Loans'].values[0])
        feature['properties']['Population'] = (
            int(city_row['Total_Population'].values[0]) if not np.isnan(city_row['Total_Population'].values[0]) else "Not Available"
        )
        feature['properties']['Median_household_income'] = float(city_row['Median_household_income'].values[0])
    else:
        # Default values for cities not in the funding data
        feature['properties']['Funding'] = 0.0
        feature['properties']['Population'] = "Not Available"
        feature['properties']['Median_household_income'] = "Not Available"

# Create the base map
ma_map = folium.Map(location=[42.4072, -71.3824], zoom_start=8)

# Create a Choropleth layer for median household income
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    data=city_stats,
    columns=["City", "Median_household_income"],
    key_on="feature.properties.census_name",
    fill_color="YlGn",  # Shades of green
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Median Household Income (USD)",
    threshold_scale=income_bins,  # Use calculated bins
).add_to(ma_map)

# Add tooltips with formatted data
tooltip = GeoJsonTooltip(
    fields=["census_name", "Funding", "Population", "Median_household_income"],
    aliases=["City:", "Funding (USD):", "Population:", "Median Household Income (USD):"],
    localize=True,
    sticky=True,
    labels=True,
    style=(
        "background-color: white; "
        "border: 1px solid black; "
        "border-radius: 3px; "
        "padding: 5px;"
    ),
    missing="Not Available",
)

# Add GeoJSON layer for interactivity and coloring
folium.GeoJson(
    geojson_data,
    style_function=lambda x: {
        'fillColor': '#f7fcf5' if x['properties']['Median_household_income'] == "Not Available" else
                     '#d9f0a3' if income_bins[0] <= x['properties']['Median_household_income'] < income_bins[1] else
                     '#addd8e' if income_bins[1] <= x['properties']['Median_household_income'] < income_bins[2] else
                     '#78c679' if income_bins[2] <= x['properties']['Median_household_income'] < income_bins[3] else
                     '#31a354' if income_bins[3] <= x['properties']['Median_household_income'] < income_bins[4] else
                     '#006837',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=tooltip
).add_to(ma_map)

# Display the map
display(ma_map)

# Sort cities by Median Household Income in descending order for debugging or further analysis
sorted_city_stats = city_stats.sort_values(by='Median_household_income', ascending=False).reset_index(drop=True)
print(sorted_city_stats[['City', 'Funding Amount Excluding Loans', 'Total_Population', 'Median_household_income']])




## Heat map by city clustering

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import folium
import requests
from folium.features import GeoJsonTooltip
from IPython.display import display
import numpy as np


df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

# Group by 'City' to get total funding amount, total population, and White population for each city
city_stats = df.groupby('City').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max',
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Add percentage of population identified as White
city_stats['Percentage_White'] = city_stats['White_Alone'] / city_stats['Total_Population']

# Features to cluster on
data = city_stats[['Percentage_White', 'Median_household_income']]

# Standardize  data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Determine optimal number of clusters using the elbow method
wcss = []
max_clusters = 10
for i in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(data_scaled)
    wcss.append(kmeans.inertia_)

################ ELbow curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, max_clusters + 1), wcss, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.show()

# Apply K-means clustering with the optimal number of clusters
optimal_clusters = 3
kmeans_optimal = KMeans(n_clusters=optimal_clusters, random_state=42)
city_stats['Cluster'] = kmeans_optimal.fit_predict(data_scaled)

# Define consistent cluster colors and labels
cluster_colors = {
    0: '#1f77b4',  # Blue
    1: '#ff7f0e',  # Orange
    2: '#2ca02c',  # Green
    -1: 'gray'     # Gray for missing data
}
cluster_labels = {
    0: "High Income and White",
    1: "Low Income and Non-White",
    2: "Low Income and White",
    -1: "Missing Data"
}

# Map cluster labels to the city_stats DataFrame
city_stats['Cluster_Label'] = city_stats['Cluster'].map(cluster_labels)

# Plot clusters
plt.figure(figsize=(10, 6))
for cluster_id, label in cluster_labels.items():
    cluster_data = city_stats[city_stats['Cluster'] == cluster_id]
    plt.scatter(
        cluster_data['Percentage_White'],
        cluster_data['Median_household_income'],
        label=f"{label} (Cluster {cluster_id})",
        alpha=0.6,
    )

# Mkae cluster graph
plt.xlabel('Percentage of Population that Identifies as White (%)')
plt.ylabel('Median Household Income (USD)')
plt.title(f'K-means Clustering of Cities in Massachusetts (k={optimal_clusters})')
plt.legend(title="Cluster Groups")
plt.grid(alpha=0.3)
plt.show()

#######################################################
### Heat Map
# Fetch the GeoJSON file
geojson_url = "https://raw.githubusercontent.com/nblmc/massachusetts-municipal-boundaries/main/data-cooked/mass-municipalities.geojson"
response = requests.get(geojson_url)
geojson_data = response.json()

# Merge cluster data into GeoJSON
for feature in geojson_data['features']:
    city_name = feature['properties']['census_name']
    city_row = city_stats[city_stats['City'].str.lower() == city_name.lower()]

    if not city_row.empty:
        # Populate properties from clustering data
        feature['properties']['Cluster'] = int(city_row['Cluster'].values[0])
        feature['properties']['Cluster_Label'] = cluster_labels[int(city_row['Cluster'].values[0])]
        feature['properties']['Funding'] = float(city_row['Funding Amount Excluding Loans'].values[0])
        feature['properties']['Population'] = (
            int(city_row['Total_Population'].values[0]) if not np.isnan(city_row['Total_Population'].values[0]) else "Not Available"
        )
        feature['properties']['Funding_Per_Capita'] = float(
            city_row['Funding Amount Excluding Loans'].values[0] / city_row['Total_Population'].values[0]
        ) if city_row['Total_Population'].values[0] > 0 else 0.0
        feature['properties']['Percentage_White'] = float(city_row['Percentage_White'].values[0])
        feature['properties']['Median_household_income'] = float(city_row['Median_household_income'].values[0])
    else:
        # Default values for cities not in the clustering data
        feature['properties']['Cluster'] = -1  # -1 for missing data
        feature['properties']['Cluster_Label'] = cluster_labels[-1]
        feature['properties']['Funding'] = 0.0
        feature['properties']['Population'] = "Not Available"
        feature['properties']['Funding_Per_Capita'] = 0.0
        feature['properties']['Percentage_White'] = "Not Available"
        feature['properties']['Median_household_income'] = "Not Available"

# Create the base map
ma_map = folium.Map(location=[42.4072, -71.3824], zoom_start=8)

# GeoJSON layer for interactivity and coloring based on clusters
folium.GeoJson(
    geojson_data,
    style_function=lambda x: {
        'fillColor': cluster_colors[x['properties']['Cluster']],
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=GeoJsonTooltip(
        fields=[
            "census_name", "Cluster", "Cluster_Label", "Funding", "Population", "Funding_Per_Capita", 
            "Percentage_White", "Median_household_income"
        ],
        aliases=[
            "City:", "Cluster ID:", "Cluster Name:", "Funding (USD):", "Population:", "Funding Per Capita (USD):",
            "Percentage White:", "Median Household Income (USD):"
        ],
        localize=True,
        sticky=True,
        labels=True,
        style=(
            "background-color: white; "
            "border: 1px solid black; "
            "border-radius: 3px; "
            "padding: 5px;"
        ),
        missing="Not Available",
    )
).add_to(ma_map)

# Display the map
display(ma_map)

# Sort cities by Cluster for debugging or further analysis
sorted_city_stats = city_stats.sort_values(by='Cluster').reset_index(drop=True)
print(sorted_city_stats[['City', 'Cluster', 'Cluster_Label', 'Funding Amount Excluding Loans', 'Total_Population', 'Percentage_White', 'Median_household_income']])



# Sorted stats for cities

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove Whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)


# Ensure all columns print side by side
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

# Group by 'City' to get total funding amount, total population, and White population for each city
city_stats = df.groupby('County').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max', 
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Calculate funding per capita
city_stats['Funding_Per_Capita'] = city_stats['Funding Amount Excluding Loans'] / city_stats['Total_Population']

# Calculate percentage of White population
city_stats['Percentage_White'] = (city_stats['White_Alone'] / city_stats['Total_Population']) * 100

# Sort cities by Funding_Per_Capita in descending order
sorted_city_stats = city_stats.sort_values(by='Median_household_income', ascending=False).reset_index(drop=True)


print(sorted_city_stats[['City', 'Funding_Per_Capita', 'Percentage_White', 'Median_household_income']])


# Scatter Plots

## percent white scatter

This code creates a scatter plot comparing the funding per capita to percent of the city's population that identifies as White. Note - only cities with at least 5000 people were included in this comparison. There is no obvious correlation between the percent White of a city and its funding per capita with a correlation coefficient of -0.14.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove Whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

# Group by 'City' to get total funding amount, total population, and White population for each city
city_stats = df.groupby('County').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max', 
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Only look at cities with at least 5000 people and Funding Per Capita below 3000 to remove outliers
population_threshold = 5000
funding_cap = 3000

# Filter out cities by thresholds
filtered_city_stats = city_stats[(city_stats['Total_Population'] >= population_threshold) & (city_stats['Funding_Per_Capita'] < funding_cap)]

# Find correlation from filtered data
filtered_correlation = filtered_city_stats['Funding_Per_Capita'].corr(filtered_city_stats['Percentage_White'])

# Calculate the line of best fit using numpy
X = filtered_city_stats['Percentage_White']
y = filtered_city_stats['Funding_Per_Capita']

# Perform linear regression
slope, intercept = np.polyfit(X, y, 1)

# Generate the regression line
line_of_best_fit = slope * X + intercept

# Make scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6)
plt.plot(X, line_of_best_fit, color='blue', linewidth=3, label='Regression Line')
plt.xlabel('Percentage of population identifying as White (%)', fontsize=14)  # Increase font size for x-axis
plt.ylabel('Funding Per Capita (USD)', fontsize=14)  # Increase font size for y-axis
plt.title('Funding Per Capita vs Percentage of population identifying as White for cities in Massachusetts')

# Add correlation on the plot
plt.text(0.75, 0.95, f'Correlation: {filtered_correlation:.2f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))

plt.show()

## Median household income scatter

Look at funding per captia vs median housing income. Holyoke was a major outlier so it was removed by adding a funding_cap of $3000 per capita. This however only changed the correlation coefficiten from -0.30 to -0.29 suggesting a moderate negative relationship.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove Whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

# Group by 'City' to get total funding amount, total population, and White population for each city
city_stats = df.groupby('County').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max', 
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Only look at cities with at least 5000 people and Funding Per Capita below 3000 to remove outliers
population_threshold = 5000
funding_cap = 3000

# Filter out cities by threshold
filtered_city_stats = city_stats[(city_stats['Total_Population'] >= population_threshold) & (city_stats['Funding_Per_Capita'] < funding_cap)]

# Calculate the correlation on the filtered data
income_correlation = filtered_city_stats['Funding_Per_Capita'].corr(filtered_city_stats['Median_household_income'])

# Calculate the line of best fit using numpy
X = filtered_city_stats['Median_household_income']
y = filtered_city_stats['Funding_Per_Capita']

# Perform linear regression
slope, intercept = np.polyfit(X, y, 1)

# Generate the regression line
line_of_best_fit = slope * X + intercept

# MAke scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(filtered_city_stats['Median_household_income'], filtered_city_stats['Funding_Per_Capita'], alpha=0.6)
plt.plot(X, line_of_best_fit, color='blue', linewidth=3, label='Regression Line')
plt.xlabel('Median Household Income (USD)', fontsize=14)  # Increase font size for x-axis
plt.ylabel('Funding Per Capita (USD)', fontsize=14)  # Increase font size for y-axis
plt.title('Funding Per Capita vs Median Household Income for cities in Massachusetts ')

# Add correlation to the plot
plt.text(0.75, 0.95, f'Correlation: {income_correlation:.2f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))

plt.show()

# Clustering

CLustering based on % White and Median household income. Of note - there are poor white towns, rich white towns, poor non-white towns but virtually no rich non-white towns.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove Whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

# Group by 'City' to get total funding amount, total population, and White population for each city
city_stats = df.groupby('County').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max', 
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()


data = city_stats[['Percentage_White', 'Median_household_income']]

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Use the elbow method to find the optimal number of clusters
# We can always adjust number of clusters depending on graph
wcss = []
max_clusters = 10 \

for i in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(data_scaled)
    wcss.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, max_clusters + 1), wcss, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.show()

# Update based on the elbow point from the plot
optimal_clusters = 3

# K-means clustering
kmeans_optimal = KMeans(n_clusters=optimal_clusters, random_state=42)
city_stats['Cluster'] = kmeans_optimal.fit_predict(data_scaled)

# Plot clusters
plt.figure(figsize=(10, 6))
plt.scatter(city_stats['Percentage_White'], city_stats['Median_household_income'], c=city_stats['Cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Percentage of Population that identifies as White (%)')
plt.ylabel('Median Household Income (USD)')
plt.title(f'K-means Clustering of Cities in Mass (k={optimal_clusters})')
plt.colorbar(label='Cluster')
plt.show()

# Bar chart

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
df = pd.read_csv(r"https://raw.githubusercontent.com/DazzedUpDas/Markey-Dataset/main/new_data.csv")
# Remove Whitespace to clean data
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

# Group by 'City' to get total funding amount, total population, and White population for each city
city_stats = df.groupby('County').agg({
    'Funding Amount Excluding Loans': 'sum',
    'Total_Population': 'max', 
    'White_Alone': 'max',
    'Median_household_income': 'max'
}).reset_index()

# Strip column names to remove any leading or trailing spaces
df.columns = df.columns.str.strip()

# Check if the columns 'County' and 'Funding Amount Excluding Loans' exist
if "County" in df.columns and "Funding Amount Excluding Loans" in df.columns:
    # Drop rows with NaN values in the 'County' and 'Funding Amount Excluding Loans' columns
    df = df.dropna(subset=["County", "Funding Amount Excluding Loans"])

    # Convert the 'Funding Amount Excluding Loans' column to numeric, forcing errors to NaN (ignoring non-numeric values)
    df['Funding Amount Excluding Loans'] = pd.to_numeric(df['Funding Amount Excluding Loans'].replace(r'[\$,]', '', regex=True), errors='coerce')

    # Group by 'County' and sum up the 'Funding Amount Excluding Loans'
    county_funding = df.groupby('County')['Funding Amount Excluding Loans'].sum().reset_index()

    # Calculate the percentage of loans for each county
    county_funding['Percentage'] = (county_funding['Funding Amount Excluding Loans'] / county_funding['Funding Amount Excluding Loans'].sum()) * 100

    # Print the result
    print("Percentage of grant funding by county:")
    print(county_funding)

    # Plotting a horizontal bar chart for the percentage of loans by county
    plt.figure(figsize=(12, 8))
    plt.barh(county_funding['County'], county_funding['Percentage'], color='skyblue')
    plt.xlabel('Percentage of Total Funding (%)')
    plt.ylabel('County')
    plt.title('Percentage of Grant Funding by County')
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    # Show the bar chart
    plt.tight_layout()
    plt.show()

else:
    print("One or both of the specified columns do not exist in the DataFrame.")

# Zipcodes

Zip Codes

In [ ]:
import geopandas as gpd

# Download data here
# https://www.mass.gov/info-details/massgis-data-zip-codes-5-digit-from-here-navteq

# Specify the path to the shapefile (without the .shp extension)
shapefile_path = r"C:\BU_MSDS\DS701\Markey\zipcodes_nt\ZIPCODES_NT_POLY"

# Read the shapefile using geopandas
try:
    gdf = gpd.read_file(f"{shapefile_path}.shp")

    # Display the first few rows of the GeoDataFrame
    print("Contents of the GeoDataFrame:")
    print(gdf.to_string(index=False))

    # Check for a column that might contain ZIP codes
    if 'POSTCODE' in gdf.columns:  # Replace 'ZIP_CODE' with the actual column name if different
        zip_codes = gdf['POSTCODE'].unique()  # Get unique ZIP codes
        print("Found ZIP Codes:")
        for code in zip_codes:
            print(code)
    else:
        print("No ZIP_CODE column found in the shapefile.")

    # Save the attribute data (without geometry) to CSV
    csv_path = "C:\BU_MSDS\DS701\Markey\zipcodes_nt\zipcodes.csv"  # New path with a .csv extension
    gdf.drop(columns='geometry').to_csv(csv_path, index=False)

except FileNotFoundError:
    print("Shapefile not found. Please check the path.")
except Exception as e:
    print("An unexpected error occurred:", str(e))

finally:
    # You can add code here that will always execute, 
    # regardless of exceptions (e.g., closing files)
    pass  # For now, we have nothing to add here

print("Processing complete.")   

## zipcode convert csv

In [ ]:
file_path = r"https://raw.githubusercontent.com/dstrick17/DS701/refs/heads/main/zipcodes.csv"
df = pd.read_csv(file_path)

print(df.head())